## Setup

In [4]:
import sys
from pathlib import Path

from aocd import get_data, submit

current_day is only available in December (EST)


In [5]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [6]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=13)

## Part a

In [8]:
# Imports
import re
from itertools import product

In [13]:
# Functions
def parse_line(string: str) -> complex:
    """Parse button or prize coordinates into complex number."""
    pattern = r"Prize: X=(\d+), Y=(\d+)" if "Prize" in string else r"Button [AB]: X\+(\d+), Y\+(\d+)"

    if not (match := re.fullmatch(pattern, string)):
        err_msg = f"Invalid string: {string}"
        raise ValueError(err_msg)

    return complex(int(match.group(1)), int(match.group(2)))

In [213]:
# Brute force solution
def find_cheapest_solution_cost(button_a: complex, button_b: complex, prize: complex) -> int:
    """Find the cheapest combination of button presses to reach the prize location.

    If no solution is found, return None.
    """
    solutions = set()

    # Try all combinations of button presses (with a cap of 100 per button)
    for a, b in product(range(1, 101), repeat=2):
        location = a * button_a + b * button_b
        if location == prize:
            cost = 3 * a + b  # Pressing button A costs 3 tokens, button B costs 1 token
            solutions.add((a, b, cost))

    if not solutions:
        return 0

    # Find cheapest solution
    return min(solution[2] for solution in solutions)

In [14]:
# Unpack data

# Add newline to data to ensure equal size of text groups
text_groups = zip(*[iter((data + "\n\n").splitlines())] * 4, strict=True)

# Parse lines into complex numbers for each claw machine
claw_machines = [tuple(map(parse_line, group[:3])) for group in text_groups]

In [217]:
# Calculate the total cost of the cheapest solution for each claw machine, if one exists
total_cost = sum(find_cheapest_solution_cost(*claw_machine) for claw_machine in claw_machines)

In [ ]:
# Submit answer
submit(total_cost, part="a", day=13, year=2024)

## Part b

In [165]:
# Imports
import numpy as np

In [285]:
# Add 10_000_000_000_000 to each prize location
claw_machines_part_b = [
    (button_a, button_b, prize + 10_000_000_000_000 * (1 + 1j)) for button_a, button_b, prize in claw_machines
]

In [342]:
# Eigenvalue solution
def find_cheapest_solution_cost_with_numpy(button_a: complex, button_b: complex, prize: complex) -> int:
    """Find the cheapest combination of integer button presses to reach the prize location.

    If no solution is found, return None.
    """
    # Create a linear basis from the button vectors
    basis = np.array([[button_a.real, button_b.real], [button_a.imag, button_b.imag]])
    prize_vector = np.array([prize.real, prize.imag])

    # Find result vector (a, b) such that a * button_a + b * button_b = prize
    result = np.round(np.linalg.solve(basis, prize_vector))

    # Check if the result is correct when rounding to integers
    if (prize_vector == result @ basis.T).all():
        return int(np.round(result) @ (3, 1))

    return 0

In [343]:
total_cost_b = sum(find_cheapest_solution_cost_with_numpy(*claw_machine) for claw_machine in claw_machines_part_b)

In [ ]:
# Submit answer
submit(total_cost_b, part="b", day=13, year=2024)